<a href="https://colab.research.google.com/github/marceloBilbyIA/transferLearningCatsAndDogs/blob/main/Transfer_Learning_Cats_and_Dogs_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTS

In [ ]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.io import read_file
import tensorflow as tf
import glob

DOWNLOAD

In [ ]:
!echo "Downloading Cat and Dogs for image notebooks"
!curl -L -o kagglecatsanddogs_5340.zip --progress-bar https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
!rm -rf PetImages
!unzip -q kagglecatsanddogs_5340.zip
!rm -rf kagglecatsanddogs_5340.zip
!mkdir -p PetImages/train
!mkdir -p PetImages/validation
!mkdir -p PetImages/validation/Cat
!mkdir -p PetImages/validation/Dog
!mv -v -f -t PetImages/train PetImages/Cat
!mv -v -f -t PetImages/train PetImages/Dog
!rm -rf CDLA-Permissive-2.0.pdf
!rm -rf readme\[1\].txt

############################################################################################# 100.0%
renamed 'PetImages/Cat' -> 'PetImages/train/Cat'
renamed 'PetImages/Dog' -> 'PetImages/train/Dog'


REMOVING CORRUPT IMAGES

In [ ]:
deleted = 0
base_dir = "PetImages"
folders = ["train/Cat/*.jpg", "train/Dog/*.jpg"]
for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    for image in sorted(glob.glob(folder_path)):
      try:
          img = read_file(str(image))
          img = tf.io.decode_image(img, channels=3)
          if img.shape[2] != 3:
             print(image)
             os.remove(image)
      except Exception as e:
            print(f"Erro ao processar {image}: {e}")
            os.remove(image)  # Opcional: remova imagens corrompidas
            deleted += 1

Erro ao processar PetImages/train/Cat/10404.jpg: {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required. [Op:DecodeImage] name: 
PetImages/train/Cat/10501.jpg
PetImages/train/Cat/10820.jpg
PetImages/train/Cat/11210.jpg
PetImages/train/Cat/11565.jpg
PetImages/train/Cat/11874.jpg
PetImages/train/Cat/11935.jpg
PetImages/train/Cat/140.jpg
PetImages/train/Cat/2663.jpg
PetImages/train/Cat/3300.jpg
PetImages/train/Cat/3491.jpg
Erro ao processar PetImages/train/Cat/4351.jpg: {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input size should match (header_size + row_size * abs_height) but they differ by 2 [Op:DecodeImage] name: 
PetImages/train/Cat/4833.jpg
PetImages/train/Cat/5553.jpg
PetImages/train/Cat/660.jpg
Erro ao processar PetImages/train/Cat/666.jpg: {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} I

DATASET

In [ ]:
base = "/content/PetImages"
train = os.path.join(base, "train")
validation = os.path.join(base, "validation")

PRE PROCCESSING

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 24940 images belonging to 2 classes.


In [ ]:
val_generator = validation_datagen.flow_from_directory(
    validation,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 4 images belonging to 2 classes.


ARCHITECTURE VGG16

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


MODEL

In [ ]:
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=output)

COMPILE

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

TRAIN MODEL

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
471/780 ━━━━━━━━━━━━━━━━━━━━ 1:31:31 18s/step - accuracy: 0.8542 - loss: 0.3278

ASSESSMENT MODEL

In [ ]:
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}")

PREDICTION

In [ ]:
def predict_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Expande a dimensão para batch único

    prediction = model.predict(img_array)

    if prediction[0] < 0.5:
        print("É um Gato!")
    else:
        print("É um Cachorro!")

EXECUTION

In [ ]:
predict_image("/content/ImagensParaTeste/gatos4.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 841ms/step
É um Gato!


In [ ]:
print(history.history.keys())

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])


In [ ]:
print(history.history)

{'accuracy': [0.8288692831993103], 'loss': [0.3989183008670807], 'val_accuracy': [1.0], 'val_loss': [0.054208554327487946]}


VISUALIZATION

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
